In [1]:
#!pip install selenium
#!pip install geonamescache
from helper import *
import selenium
import tweepy
import numpy as np
import pandas as pd
import json
from bs4 import BeautifulSoup
from IPython.display import clear_output #clears output
import datetime
import time

API_Key ='irXxqVrXin1KsDRM9g6HdF8p8'
API_Key_Secret = 'sLQvb8voyrTDz5S93vlt5jE5RlMi8xBuOQtSq2pgERe2QVawIC'

Bearer_Token = 'AAAAAAAAAAAAAAAAAAAAACsVlAEAAAAADib49u9Asa6Hbts8UwBEdtc0dsk%3D1d8lVaySeRg1CpJQ6HDLSwK6Mj9wJiptj49PAdwMuvW95zJoar'

Access_Token = '1122492746210660352-COptyz3RtF1AVokHw3027iOdbsZZl1'
Access_Token_Secret = 'slfBz8pcgCrUmewZgSgVKsmIFhfCcFuinAx16q5eh1Vvr'

In [2]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(API_Key, API_Key_Secret)
# set access to user's access key and access secret
auth.set_access_token(Access_Token, Access_Token_Secret)
# calling the api
api = tweepy.API(auth)

Client = tweepy.Client(bearer_token=Bearer_Token) 


### retrieving the tweets
we will begin by saving the WOEIDs of each country in variables(The WOEID is a 32 bit adress of a place on earth, it's an odd system twitter adopted for some reason)

we will then ask the API to get us 2000 tweets from each WOEID.
we do this by giving the API a spacial query:
"plase:" + WOEID = tells the API to only retrieve tweets from that location

"-filter:replies" = tells the API to ignore tweets that are actually replies
we ignore them since replies rarely get retweets and they basically filled our DataFrame with a lot of replies that have 0 retweets.

"-filter:retweets" = we removed this since apperantly whenever someone retweets a tweets, it's concidered by twitter as it's own tweet. so what ended up happening is that we had several of the same tweet show up, it filled our Data Frame with duplicated.

afterwards we activate a function we wrote called "getAllTrends()"
the function uses the lenium to crawl into 'www.trends24.in' and get the trending topics in each country and every city in it(that is avalible)

twitter's API has a function that allows us to get the trends in each area, however it has a very limited rate. So we used a third party website instead.

In [3]:
'''
#tweets_list = api.search_30_day(label = '30day',query = '#blender', fromDate = FROM_DATE,toDate = TO_DATE,maxResults = 10,tweet_mode = "extended")
#tweets_list = tweepy.Cursor(api.search_tweets, "biden", count=100).items(100)


start_date = datetime.datetime.now() - datetime.timedelta(days=14)
end_date = start_date + datetime.timedelta(days=7)

start_date = datetime.datetime.now()
end_date = start_date + datetime.timedelta(days=1)
'''

# Search for tweets within the target countries
usa = '96683cc9126741d1'
canada = "3376992a082d67c7"
great_britain = '6416b8512febefc9'
Australia = '3f14ce28dc7c4566'
new_zealand = "0b039ce18ddf1cb9"

tweets_list = []
#ALL_pages = tweepy.Cursor(api.search_tweets, q="since:{} until:{}".format(start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d")), lang="en", tweet_mode="extended", count = 100).pages(50)
max_count_4_each_country = 2000

#add usa
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ usa + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('Us secured')
time.sleep(10)

#add Canada
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ canada + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('canada secured')
time.sleep(10)

#add "Great" Britian
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ great_britain + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('UK secured')
time.sleep(10)

#add Australia
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ Australia + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print('kengooro land secured')
time.sleep(10)

#add new zealand
tweets_list += tweepy.Cursor(api.search_tweets, q="place:"+ new_zealand + ' -filter:replies -filter:retweets', tweet_mode="extended",include_entities=True,count = 100).items(max_count_4_each_country)
print("kengooroon't land secured")


'''
#return this if you are using pages again!!!!!
for page in ALL_pages:
    tweets_list.extend(page)
#tweets_list = [t for t in tweets_list if t.in_reply_to_status_id == None]    
'''

trends = getAllTrends()#get trends

print('done')

Us secured
canada secured
UK secured
kengooro land secured
kengooroon't land secured
done


In [4]:
tweets_list = [t for t in tweets_list]
len(tweets_list)

10000

### crawling
now the interseting part.

firstly we will extract everything we need from each tweet
1) the text
2) the amount of words
3) how many likes the tweet has
4) how many retweets the tweet has
5) is it a quote tweet
6) the date the tweet was posted
7) the ID of the tweet(this will be useful in the reextraction)
8) the user's information

then we will go over all users we have:
because the twitter API likes to hide literally anything useful we will have to extract some information with brute force
so we will use selenium to enter each users profile page

using selenium and the API we extract he following:
1) the screen name of the user(the name that is eunique to that spacific user)
2) how many followers the user has
3) how many people the user follows
4) when the user joined(account age)
5) how many tweets the user has tweeted in general
6) is the user verified
7) gender

the API didn't give us the gender and age of the user.
so we crawled into their profile and extracted it.

in twitter it is very common to put the user's pronounce in their name,bio and or even location
so we searched there


In [5]:
#search_tweets(q, *, geocode, lang, locale, result_type, count, until, since_id, max_id, include_entities)

#user data

DRIVER = webdriver.Chrome()#the chrome

user_name = [];
#location = [];

city = [];
country = [];
age = [];
gender = [];
followers = [];
following = [];
verified = []
joined = [] #when they joined
acc_age = [] 
respectability = []
tweet_count_user = []

#tweet data
tweet = [];
word_count = [];
likes = []
retweet_count = []
is_quote_status = []
date = []
tweet_id = []
hashtags = []
hashtags_count = []
trending_for_area = []
is_using_trend = []
users = [];

print('loading users')

arr = []
i = 1

for t in tweets_list:
    tweet.append(t.full_text);
    
    users.append(t.user);
    
    word_count.append(countWordsInString(t.full_text))
    
    likes.append( t.favorite_count)
    
    retweet_count.append(t.retweet_count)
    
    is_quote_status.append(t.is_quote_status)
    
    date.append(t.created_at.isoformat()[:10])
    
    tweet_id.append(t.id)
    
    
        
    city.append(t.place.name)
    country.append( t.place.country)
    
    loc_trends = getTrendsByLoc(t.place.name,t.place.country,trends)
    trending_for_area.append(loc_trends)
    is_using_trend.append(areTrendUsed(t.full_text,loc_trends))
    
    ht = [t['text'] for t in t.entities['hashtags']]
    ht_count = len(ht)
    
    if ht == []:
        ht = np.nan
        ht_count = 0
        
    hashtags.append(ht)
    hashtags_count.append(ht_count)
    
    
    
    
    
        
for x in users:
    if i%26 == 0:
        print("waiting")
        time.sleep(6)
    j = x._json
    
    #opens the page
    screen_name = j['screen_name']    
    DRIVER.get('https://twitter.com/' + screen_name) 
    
    #get the easy stuff
    user_name.append(screen_name)
    following.append(j['friends_count'])
    followers.append(j['followers_count'])
    
    if(j['followers_count'] != 0):
        respectability.append(float(j['friends_count']) / float(j['followers_count']))
    else:
        respectability.append(0)
        
    joined.append(x.created_at.isoformat()[:10])
    acc_age.append(2023 - x.created_at.year + float(x.created_at.day)/12)
    tweet_count_user.append(x.statuses_count)
    verified.append(x.verified)
    
    
    
    tmp = getAge(DRIVER)#gets the age
    if(tmp == 2023 - x.created_at.year): #checks if the age is the same as the accounts age
        age.append(np.nan)
    else:
        age.append(tmp)
    
    
    gender.append(getGender(j['description'],j['location'],j['name']))
    
    ##this part simply displays at the bottom how many profiles were checked
    clear_output(wait=True) #clears the output
    
    i+=1
    print(i)

print('done')

9800


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=109.0.5414.74)
Stacktrace:
Backtrace:
	(No symbol) [0x00A6F243]
	(No symbol) [0x009F7FD1]
	(No symbol) [0x008ED04D]
	(No symbol) [0x008DFC24]
	(No symbol) [0x008DFA5C]
	(No symbol) [0x008DE7E8]
	(No symbol) [0x008DEEF7]
	(No symbol) [0x008E7F08]
	(No symbol) [0x008F3455]
	(No symbol) [0x008F6766]
	(No symbol) [0x008DF2A1]
	(No symbol) [0x008F328F]
	(No symbol) [0x0094C8F2]
	(No symbol) [0x00938386]
	(No symbol) [0x0091163C]
	(No symbol) [0x0091269D]
	GetHandleVerifier [0x00D09A22+2655074]
	GetHandleVerifier [0x00CFCA24+2601828]
	GetHandleVerifier [0x00B18C0A+619850]
	GetHandleVerifier [0x00B17830+614768]
	(No symbol) [0x00A005FC]
	(No symbol) [0x00A05968]
	(No symbol) [0x00A05A55]
	(No symbol) [0x00A1051B]
	BaseThreadInitThunk [0x757500F9+25]
	RtlGetAppContainerNamedObjectPath [0x77317BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77317B8E+238]


In [6]:
max = len(gender)

for i in range(0,max):#rounds the account ages
    acc_age[i] = round(acc_age[i],1)

In [7]:
metaData=['--------' for i in range(max)]
dick = {"name":user_name[:max],
        "age":age[:max],
        'city':city[:max],
        'country':country[:max],
        "gender":gender[:max],
        'acount age':acc_age[:max],
        'total tweets': tweet_count_user[:max],
        "followers":followers[:max],
        "following":following[:max],
        'respectability':respectability[:max],
        'verified' : verified[:max],
        'tweet':tweet[:max],
        'likes':likes[:max],
        'retweets':retweet_count[:max],
        'word count': word_count[:max],
        'is quote':is_quote_status[:max],
        'hashtags':hashtags[:max],
        'hashtag count':hashtags_count[:max],
        'trending': trending_for_area[:max],
        'using trends?':is_using_trend[:max],
        'metaData':metaData[:max],
        'join_date':joined[:max],
        'date':date[:max],
        'tweet id':tweet_id[:max]
       }

df = pd.DataFrame(dick)

In [8]:
df

name  age                   city        country  gender  \
0       streetsforall  NaN     Desert Hot Springs  United States     NaN   
1            Quersha_  NaN              Charlotte  United States     NaN   
2     Gatitaconestres  NaN          New Hampshire  United States     NaN   
3         Cheli_Smith  NaN       Upstate New York  United States     NaN   
4         IgawaPastor  NaN          Mission Viejo  United States     NaN   
...               ...  ...                    ...            ...     ...   
9794  arshadzackeriya  NaN        Wellington City    New Zealand     NaN   
9795        Ncookie98  NaN               Auckland    New Zealand     NaN   
9796          erimedi  NaN      Christchurch City    New Zealand     NaN   
9797      auralorgasm  NaN  Kapiti Coast District    New Zealand     NaN   
9798        Wiki_Pita  NaN               Auckland    New Zealand     NaN   

      acount age  total tweets  followers  following  respectability  ...  \
0           13.5          4714       8223        137        0.016661  ...   
1           14.5        105862       1239        480        0.387409  ...   
2            5.0         12600        333        189        0.567568  ...   
3           12.8         13567        542       2121        3.913284  ...   
4           10.2          5113        150        398        2.653333  ...   
...          ...           ...        ...        ...             ...  ...   
9794        14.0          1022        289        482        1.667820  ...   
9795        11.3        133568        121        516        4.264463  ...   
9796         8.0         10015        190        376        1.978947  ...   
9797         9.8         56050        490        412        0.840816  ...   
9798         9.3         21218       1970       3482        1.767513  ...   

      word count is quote                                           hashtags  \
0             34     True                                                NaN   
1             10    False                                                NaN   
2              4     True                                                NaN   
3             17    False  [shortfilmmaking, onset, cinema, tvfilm, movie...   
4              9    False                                                NaN   
...          ...      ...                                                ...   
9794          26    False                           [DevOps, DevOpswithZack]   
9795           2    False                                                NaN   
9796          11    False                                                NaN   
9797           4     True                                                NaN   
9798          11     True                                                NaN   

      hashtag count                                           trending  \
0                 0  [#TabooToken, Chargers, Herbert, Staley, Trevo...   
1                 0  [#TabooToken, Chargers, Good Sunday, Herbert, ...   
2                 0  [#TabooToken, Chargers, Herbert, Staley, Trevo...   
3                 7  [#TabooToken, Chargers, Herbert, Staley, Trevo...   
4                 0  [#TabooToken, Chargers, Herbert, Staley, Trevo...   
...             ...                                                ...   
9794              2  [Paula, Chargers, Paul Henry, #UFCVegas67, #su...   
9795              0  [Paula, Chargers, Paul Henry, Perth, #UFCVegas...   
9796              0  [Paula, Chargers, Paul Henry, #UFCVegas67, #su...   
9797              0  [Paula, Chargers, Paul Henry, #UFCVegas67, #su...   
9798              0  [Paula, Chargers, Paul Henry, Perth, #UFCVegas...   

      using trends?  metaData   join_date        date             tweet id  
0             False  --------  2010-04-06  2023-01-15  1614604722199228416  
1             False  --------  2010-05-18  2023-01-15  1614604717782437893  
2             False  --------  2019-11-12  2023-01-15  1614604717727928320  
3             False  

The crawling is finished, we can close the browser and save the DataFrame to use later

In [9]:
DRIVER.quit()

In [10]:
df.to_csv('data6.csv')